In [ ]:
!pip install pyvmomi

`SmartConnect` to a vCenter server.

Users are required to input vCenter IP and credentials.

In [ ]:
vcenter_host = '127.0.0.1'
from pyVim.connect import SmartConnect
service_instance = SmartConnect(
    host=vcenter_host,
    pwd='pass',
    user='admin'
)

Acquiring the established session's cookies:

In [ ]:
from pyVim.connect import GetStub
cookie_str = GetStub().cookie

Extracting the session ID from the `cookie_str`:

In [ ]:
import http.cookies
cookie = http.cookies.SimpleCookie(cookie_str)
session_id = cookie['vmware_soap_session'].value

Acquring the `EventManager` managed object identifier:

In [ ]:
event_manager = service_instance.content.eventManager
event_manager_moid = event_manager._moId

Specify a collector configuration with a 'filter through all' `EventFilterSpec`:

In [ ]:
event_filter_spec = '''
{
    "filter":{
        "_typeName": "EventFilterSpec"
    }
}'''

Create the collector:

In [ ]:
import requests
resp = requests.post(
    url=f'https://{vcenter_host}/sdk/vim25/8.0.1.0/EventManager/{event_manager_moid}/CreateCollectorForEvents',
    headers={'Content-type': 'application/json',
             'vmware-api-session-id': session_id},
    data=event_filter_spec)

Acquire the new collector's managed object identifier:

In [ ]:
import json
event_history_collector_moref = json.loads(resp.content)
event_history_collector_moid = event_history_collector_moref['value']

Get latest events:

In [ ]:
resp = requests.get(
    url=f'https://{vcenter_host}/sdk/vim25/8.0.1.0/EventHistoryCollector/{event_history_collector_moid}/latestPage',
    headers={'Content-type': 'application/json',
             'vmware-api-session-id': session_id})

Deserialize and inspect the latest event:

In [ ]:
events = json.loads(resp.content)
events[0]